In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models

def build_model():
    model = models.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Adversarial Examples (FGSM)
def generate_adversarial_examples(model, x, y, epsilon=0.1):
    with tf.GradientTape() as tape:
        tape.watch(x)
        predictions = model(x)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y, predictions)
    gradients = tape.gradient(loss, x)
    x_adv = x + epsilon * tf.sign(gradients)
    return tf.clip_by_value(x_adv, 0.0, 1.0)

# Tangent Distance
def tangent_distance(x, x_adv):
    return tf.linalg.norm(x - x_adv, axis=1)

# Tangent Propagation
def tangent_propagation(model, x, epsilon=0.1):
    with tf.GradientTape() as tape:
        tape.watch(x)
        predictions = model(x)
    gradients = tape.gradient(predictions, x)
    perturbation = epsilon * tf.sign(gradients)
    return perturbation

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
model = build_model()
for epoch in range(5):
    print(f"Epoch {epoch+1}/5")
    x_adv_train = generate_adversarial_examples(model, tf.convert_to_tensor(x_train, dtype=tf.float32),
                                                tf.convert_to_tensor(y_train, dtype=tf.float32), epsilon=0.1)
    x_adv_test = generate_adversarial_examples(model, tf.convert_to_tensor(x_test, dtype=tf.float32),
                                               tf.convert_to_tensor(y_test, dtype=tf.float32), epsilon=0.1)
    x_combined = tf.concat([tf.convert_to_tensor(x_train, dtype=tf.float32), x_adv_train], axis=0)
    y_combined = tf.concat([tf.convert_to_tensor(y_train, dtype=tf.int32), tf.convert_to_tensor(y_train, dtype=tf.int32)], axis=0)
    assert x_combined.shape[0] == y_combined.shape[0], f"Mismatch: {x_combined.shape[0]} != {y_combined.shape[0]}"
    model.fit(x_combined, y_combined, batch_size=128, epochs=1, verbose=0)
    perturbation = tangent_propagation(model, tf.convert_to_tensor(x_test, dtype=tf.float32))
    dist = tangent_distance(tf.convert_to_tensor(x_test, dtype=tf.float32), x_adv_test)
    print(f"  - Training loss: {model.history.history['loss'][0]:.4f}")
    print(f"  - Training accuracy: {model.history.history['accuracy'][0]:.4f}")
    print(f"  - Tangent Distance: {dist.numpy().mean():.4f}")
    print()


Epoch 1/5
  - Training loss: 0.2390
  - Training accuracy: 0.9344
  - Tangent Distance: 0.3895

Epoch 2/5
  - Training loss: 0.1814
  - Training accuracy: 0.9569
  - Tangent Distance: 0.3996

Epoch 3/5
  - Training loss: 0.1426
  - Training accuracy: 0.9649
  - Tangent Distance: 0.4062

Epoch 4/5
  - Training loss: 0.1260
  - Training accuracy: 0.9682
  - Tangent Distance: 0.4063

Epoch 5/5
  - Training loss: 0.1128
  - Training accuracy: 0.9725
  - Tangent Distance: 0.4072

